# FM 24 Moneyball

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show all columns in pandas
pd.set_option('display.max_columns', 500) 

%matplotlib inline

## Ingest and clean data

In [3]:
### Read raw data

# Select file path for raw data
path = r'C:\Users\kevmi\Documents\FM Data Lab\Man Utd\FM 24 exports\Winter 2023\Center Backs.html'

# Ingest in a dataframe
df_players_raw = pd.read_html(path, encoding='utf-8')[0]

# Format column names 
df_players_raw.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_players_raw.columns]

# ---------------------------------------------------------------------------------------------------------------------

### Read processed Data lab data

# Select file path for processed data
path = r'C:\Users\kevmi\Documents\FM 24 Moneyball\FM 24 Moneyball repo\FM-24-Moneyball\data\Man Utd Save\Winter 2023\CB\Center Backs.csv'

# Ingest in a dataframe 
df_players = pd.read_csv(path)   

# Format column names 
df_players.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_players.columns]

# Bring ID from raw data
df_players = df_players.merge(df_players_raw[['name', 'age', 'height', 'weight', 'uid']], 
                 left_on=['name', 'age', 'height', 'weight'], 
                 right_on=['name', 'age', 'height', 'weight']).reset_index().rename(columns={'level_0': 'id'})

df_players['id'] = df_players['uid']

df_players.drop(columns='uid', inplace=True)

## Data cleaning

In [4]:
df_players.head(10) 

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,tackles_attempted,tackles_attempted/90,tackles_completed,tackles_completed/90,tackles_failed,tackle_completion_%,tackles_failed/90,key_tackles,key_tackles/90,tackle_quality,interceptions,interceptions/90,blocks,blocks/90,shots_blocked,shots_blocked/90,headers_attempted,headers_attempted/90,headers_won,headers_won/90,headers_won_%,headers_lost,headers_lost/90,headers_lost_%,key_headers,key_headers/90,aerial_challenges_attempted/90,duels_win_%,fouls_against,fouls_made,net_fouls,fouls_won/90,fouls_committed/90,clearances,clearances/90,red_cards,yellow_cards,yellows/tackle,reds/tackle,yellows/90,reds/90,player_of_the_match,mistakes_leading_to_goal,sprints/90,attacking_actions/90,creative_actions/90,defensive_actions/90,goalkeeping_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister
0,29114971,Corentin Tolisso,"D (C), DM, M/AM (C)",29,181 cm,81 kg,-,Lyon,Ligue 1 Uber Eats,FRA (TOG),-,-,Scouting Required,"$99,000 p/w",$550K - $5.6M,-,Right,16,1251,7.02,0,78.19,0,30,2.16,22,1.58,8,73,0.58,2,0.14,3.75,17,1.22,7,0.50,4,0.29,52,3.74,28,2.01,53.85,24,1.73,46.15,4,0.29,3.74,60.98,7,8,1,0.50,0.58,9,0.65,0,0,0.00,0.00,0.00,0.00,0,0,15.11,1.97,1.73,12.68,0,4.53,89,100,2,80,94,97,96,97,96,98,97,99,5,0,80
1,37060952,Quinten Timber,"D (C), DM, M/AM (C)",22,177 cm,71 kg,-,Feyenoord,Eredivisie,NED (CUW),-,Spirited,"Evasive, Reserved","$18,250 p/w",$53M - $77M,-,Right,15,1325,7.05,3,73.61,1,31,2.11,28,1.90,3,90,0.20,0,0.00,4.43,26,1.77,6,0.41,1,0.07,67,4.55,27,1.83,40.30,40,2.72,59.70,2,0.14,4.55,56.12,6,7,1,0.41,0.48,7,0.48,0,1,0.03,0.00,0.07,0.00,0,0,12.29,2.02,3.06,9.03,0,5.30,98,100,4,97,99,99,96,98,98,97,98,77,3,3,99
2,91132290,Benedikt Gimber,"D (C), DM, M (C)",26,187 cm,85 kg,-,FC Heidenheim,Bundesliga,GER,-,-,Scouting Required,"$15,500 p/w",$325K - $3.3M,-,Left,18,1468,6.59,0,81.56,2,53,3.25,34,2.08,19,64,1.16,1,0.06,1.56,26,1.59,7,0.43,3,0.18,75,4.60,53,3.25,70.67,22,1.35,29.33,5,0.31,4.60,67.97,2,34,32,0.12,2.08,28,1.72,1,3,0.06,0.02,0.18,0.06,0,0,10.85,1.03,0.55,13.76,0,1.84,71,100,3,49,86,50,46,50,65,66,59,74,12,5,79
3,16096614,Florian Grillitsch,"D (C), DM, M (C)",28,187 cm,77 kg,-,TSG Hoffenheim,Bundesliga,AUT,-,-,Scouting Required,"$45,000 p/w",$9.8M - $15.5M,-,Right,13,1169,6.87,3,73.06,3,36,2.77,20,1.54,16,56,1.23,0,0.00,3.60,23,1.77,5,0.38,2,0.15,56,4.31,22,1.69,39.29,34,2.62,60.71,4,0.31,4.31,45.65,5,10,5,0.38,0.77,10,0.77,0,0,0.00,0.00,0.00,0.00,1,1,12.70,1.36,1.23,10.66,0,3.00,97,100,0,81,97,98,96,92,97,97,98,95,7,2,89
4,18077357,Dario Van Den Buijs,"D (C), DM, M (C)",28,182 cm,75 kg,-,RKC Waalwijk,Eredivisie,BEL,-,-,Scouting Required,"$1,900 p/w",$800K - $1.8M,-,Left,17,1530,6.58,0,90.00,4,22,1.29,16,0.94,6,73,0.35,6,0.35,3.14,35,2.06,11,0.65,9,0.53,97,5.71,65,3.82,67.01,32,1.88,32.99,10,0.59,5.71,68.07,2,7,5,0.12,0.41,26,1.53,0,3,0.14,0.00,0.18,0.00,0,0,3.94,0.12,0.00,17.18,0,0.29,3,100,47,12,2,12,26,16,11,7,14,29,32,39,35
5,2000028409,Raniele,"D (C), DM, M (C)",26,184 cm,78 kg,-,COR,Campeonato Brasileiro Série A Assaí,BRA,-,-,Scouting Required,"$8,250 p/w",$6.2M - $12M,-,Right,13,1199,7.04,9,54.50,5,45,3.38,35,2.63,10,78,0.75,3,0.23,2.14,25,1.88,6,0.45,3,0.23,45,3.38,31,2.33,68.89,14,1.05,31.11,4,0.30,3.38,73.33,9,21,12,0.68,1.58,13,0.98,0,1,0.02,0.00,0.08,0.00,1,0,9.38,1.62,2.10,14.98,0,4.58,99,100,24,94,99,98,93,97,96,96,95,78,6,11,95
6,29084738,Luke O'Nien,"D (C), DM, M (C)",29,177 cm,76 kg,-,Sunderland,Sky Bet Championship,ENG (SGP),Trained in nation (15-21),-,Scouting Required,"$11,250 p/w",$1.1M - $11M,-,Right,26,2295,6.90,0,88.27,6,31,1.22,25,0.98,6,81,0.24,5,0.20,1.94,53,2.08,27,1.06,

## Goalkeepers

## Center Backs

In [101]:
# Filter players that can play center back 
# We are filtering D(C), D(CL)
df_cb = df_players[df_players['position'].str.contains(r'D \([A-Z]*C[A-Z]*\)', regex=True, na=False)]

In [104]:
df_cb

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,goals,goals/90,minutes/goal,xg,xg/90,xg/shot,xg_overperformance,xg_overperformance/90,non_pen_goals,non_pen_goals/90,non_pen_goals/shot,minutes/non_pen_goal,non_pen_xg,non_pen_xg/90,non_pen_goals___non_pens_xg_/90,non_pen_xg/shot,non_pen_xg_overperformance,non_pen_xg_overperformance/90,goals_outside_box,goals_outside_box/90,assists,assists/90,minutes/assist,xa,xa/90,xa_overperformance,xa_overperformance/90,assists/clear_cut_chances_created,goal_contributions,goal_contributions/90,xgc,xgc/90,xgc_overperformance,xgc_overperformance/90,non_pen_goal_contributions,non_pen_goal_contributions/90,non_pen_xgc,non_pen_xgc/90,non_pen_xgc_overperformance,non_pen_xgc_overperformance/90,conversion_%,shots,shots/90,shots_on_target,shots_on_target/90,shots_on_target_%,shots_outside_box/90,passes_attempted,passes_attempted/90,passes_completed,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_passes,progressive_pass_rate,key_passes,key_passes/90,key_pass_%,open_play_key_passes,open_play_key_passes/90,open_play_key_pass_%,crosses_attempted,crosses_attempted/90,crosses_completed,crosses_completed/90,crosses_completed_%,open_play_crosses_attempted,open_play_crosses_attempted/90,open_play_crosses_completed,open_play_crosses_completed/90,open_play_cross_completion_%,chances_created,chances_created/90,clear_cut_chances_created,clear_cut_chances_created/90,pressures_attempted,pressures_attempted/90,pressures_completed,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,dribbles/90,dribbles,penalties_taken,penalties_scored,pens_scored_%,tackles_attempted,tackles_attempted/90,tackles_completed,tackles_completed/90,tackles_failed,tackle_completion_%,tackles_failed/90,key_tackles,key_tackles/90,tackle_quality,interceptions,interceptions/90,blocks,blocks/90,shots_blocked,shots_blocked/90,headers_attempted,headers_attempted/90,headers_won,headers_won/90,headers_won_%,headers_lost,headers_lost/90,headers_lost_%,key_headers,key_headers/90,aerial_challenges_attempted/90,duels_win_%,fouls_against,fouls_made,net_fouls,fouls_won/90,fouls_committed/90,clearances,clearances/90,offsides,offsides/90,offside/non_pen_goals,offside/non_pen_xg,distance_covered/90,distance_covered,total_saves,saves/90,save_%,xsave_%,xsave_%_overperformance,saves_held,saves_held/90,saves_held_ratio,saves_held/shots_faced_ratio,saves_tipped,saves_tipped/90,saves_tipped_ratio,saves_tipped/shots_faced_ratio,saves_parried,saves_parried/90,saves_parried_ratio,saves_parried/shots_faced_ratio,saves/goal_conceaded,save_efficiency,shots_on_target_against,shots_on_target_against/90,xgp,xgp/90,penalties_faced,penalties_saved,pens_saved_%,goals_conceded,conceded/90,clean_sheets,clean_sheet_ratio,red_cards,yellow_cards,yellows/tackle,reds/tackle,yellows/90,reds/90,player_of_the_match,mistakes_leading_to_goal,sprints/90,attacking_actions/90,creative_actions/90,defensive_actions/90,goalkeeping_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,positions_list
1371,37060952,Quinten Timber,"D (C), DM, M/AM (C)",22,177 cm,71 kg,-,Feyenoord,Eredivisie,NED (CUW),-,Spirited,"Evasive, Reserved","$18,250 p/w",$53M - $77M,-,Right,15,1325,7.05,3,73.61,1371,1,0.07,1325.00,1.80,0.12,0.09,-0.80,-0.05,1,0.07,0.05,1325.00,1.80,0.12,-0.05,0.09,-0.80,-0.05,0,0.00,4,0.27,331.25,2.85,0.19,1.15,0.08,0.80,5,0.34,4.65,0.32,0.35,0.02,5,0.34,4.65,0.32,0.35,0.02,5,19,1.29,7,0.48,37,0.75,934,63.44,821,55.77,88,7.00,103,11.03,33,2.24,3.53,33,2.24,3.53,12,0.82,3,0.20,37,8,0.54,3,0.20,37.50,11,0.75,5,0.34,145,9.85,29,1.97,20.00,10.05,

## Full Backs